# MACHINE TRANSLATION WITH FAIRSEQ (2/2) - TAMIL_TO_ENGLISH

Facebook's AI team recently released its machine translation models for Tamil<->English, and I was eager to give it a try since Tamil is among the most under-served languages in machine learning, and related language pairs are pretty hard to come by.

This is also a good opportunity to try out FB's Fairseq library, which seems pretty user-friendly for folks who work with pandas dataframes. Over two short notebooks, I'll demo a simple workflow for:

* translating English speeches and news stories to Tamil
* translating Tamil speeches and news stories to English.

## DATASET
The datasets used in these two notebooks involve some existing ones already in use by other notebooks in this repo, as well as new ones selected for this exercise.

* Existing datasets:

    - The English speeches are taken from an existing dataset comprising 11 speeches in 4 languages (English, Malay, Chinese, and Tamil) from the website of the Singapore Prime Minister's Office.
    - The English news stories (on Covid-19) were published on Singapore news outlet's CNA's website in March 2020, and were used in earlier notebooks on English-to-Chinese translation.

* New datasets:
    - The Tamil speeches (tamil_speeches.csv) are also taken from the website of the Singapore Prime Minister's Office. These are official speeches with English versions (not direct translations though) for reference.
    - The Tamil news stories (tamil_stories.csv) are taken from Sethi, Singapore state broadcaster Mediacorp's Tamil-language news website
I kept the new datasets small so as to keep the run times manageable.

## RESULTS
The output CSV files with the translated text and original copy can be [downloaded here](https://www.dropbox.com/sh/26b7q5tqyqp8kim/AAB6HGPhmrF4_Ijdr0NN8hAFa). 

There are still some obvious problems with the quality of the translation. But as mentioned in earlier notebooks, machine translation gets about 70-80% of the job done, allowing human translators to work more efficiently.

I've not done comparisons to see whether the machine translation results from fairseq are better/comparable with those from MarianMT.

This notebook focuses on Tamil-to-English translations. See notebook4.3 for English-to-Tamil translation.

In [1]:
import pandas as pd
import re
import torch
import warnings

warnings.filterwarnings('ignore')

# 1. TRANSLATE TAMIL SPEECHES TO ENGLISH

I'm unfamiliar with the tools out there for Tamil NLP, and there doesn't seem to be a Tamil-language equivalent for NLTK's sentence tokenizer in the wild. So I took the clumsy approach of splitting each sentence into a separate row for translation before concating the results back again. OK, I think, for a simple exercise like this but clearly not recommended if you need to do this in production.......

In [2]:
raw = pd.read_csv('../data/tamil_speeches.csv')

In [3]:
raw.head()

,Date,Speaker,Title,Language,Text,URL
0,2020-04-30,Lee Hsien Loong,May Day Message 2020,Tamil,"இவ்வாண்டு, நாம் சிரமமான சூழ்நிலைகளுக்கு இடையில...",https://www.pmo.gov.sg/Newsroom/PM-Lee-Hsien-L...
1,2020-08-09,S Iswaran,National Day Message 2020,Tamil,"ஒவ்வோர் ஆண்டும், மழையோ வெயிலோ, ஆகஸ்ட் 9-ஆம் தே...",https://www.pmo.gov.sg/Newsroom/National-Day-M...
2,2019-08-09,S Iswaran,National Day Message 2019,Tamil,தேசிய தினச் செய்தி 2019\nஎன் சக சிங்கப்பூரர்கள...,https://www.pmo.gov.sg/Newsroom/National-Day-M...
3,2018-08-09,S Iswaran,National Day Message 2018,Tamil,"என் சக சிங்கப்பூரர்களே, வணக்கம்.\n\nஒவ்வோர் ஆண...",https://www.pmo.gov.sg/Newsroom/national-day-m...
4,2017-08-09,S Iswaran,National Day Message 2017,Tamil,"என் சக சிங்கப்பூரர்களே,\nவணக்கம்.\n\nநான் இப்ப...",https://www.pmo.gov.sg/Newsroom/national-day-m...


## 1.2 SPLIT + CLEAN TEXT

I've used this approach as well in earlier notebooks. Essentially the new function creates a new DF with the text split into individual rows.

In [4]:
# source: https://github.com/cognoma/genes/blob/721204091a96e55de6dcad165d6d8265e67e2a48/2.process.py#L61-L95
def split_text(df, column, sep=".", keep=False):
    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            new_values.append(presplit)
        for value in values:
            indexes.append(i)
            new_values.append(value)
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    return new_df

In [5]:
def clean_text(text):
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\n\n", " ", text)
    text = text.strip(" ")
    text = re.sub(' +',' ', text).strip() # gets rid of multiple spaces and replace with a single
    return text

In [6]:
raw["Clean_Text"] = raw['Text'].map(lambda text: clean_text(text))

speeches = split_text(raw, "Clean_Text")

speeches = speeches[speeches["Clean_Text"] != ""].copy()

## 1.3 TRANSLATE

This took about 11.5 minutes on my late-2015 iMac. Prob can go much faster on the newer machines or Colab Pro.

In [7]:
ta2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt20.ta-en')

Using cache found in /Users/empire/.cache/torch/hub/pytorch_fairseq_master


In [8]:
%%time
speeches["Machine_Translation"] = speeches["Clean_Text"].apply(lambda text: ta2en.translate(text))

CPU times: user 34min 52s, sys: 5.31 s, total: 34min 58s
Wall time: 8min 51s


## 1.4 RE-ORG FOR CLARITY

The translated speeches can be "re-assembled" back using a simple groupby operation and removing the duplicates. An additional step was needed to replace the odd ".," characters after joining up the rows.

In [9]:
%%time 
speeches['English_Translation'] = speeches.groupby(["Title"])["Machine_Translation"].transform(lambda x: ','.join(x))

speeches = speeches.drop_duplicates(subset=["English_Translation"]).copy()

speeches["English_Translation"] = speeches['English_Translation'].str.replace(".,", ". ", regex=False)

CPU times: user 68.5 ms, sys: 6.72 ms, total: 75.2 ms
Wall time: 60 ms


In [10]:
cols1 = ["Date", "Speaker", "Title", "Text", "English_Translation", "URL"]

speeches_translated = speeches[cols1]

In [11]:
speeches_translated.head()

,Date,Speaker,Title,Text,English_Translation,URL
0,2020-04-30,Lee Hsien Loong,May Day Message 2020,"இவ்வாண்டு, நாம் சிரமமான சூழ்நிலைகளுக்கு இடையில...","This year, we observe May Day, amid difficult ...",https://www.pmo.gov.sg/Newsroom/PM-Lee-Hsien-L...
1,2020-08-09,S Iswaran,National Day Message 2020,"ஒவ்வோர் ஆண்டும், மழையோ வெயிலோ, ஆகஸ்ட் 9-ஆம் தே...","Every year, rain or heat, on August 9th, Singa...",https://www.pmo.gov.sg/Newsroom/National-Day-M...
2,2019-08-09,S Iswaran,National Day Message 2019,தேசிய தினச் செய்தி 2019\nஎன் சக சிங்கப்பூரர்கள...,"National Day News 2019 My fellow Singaporeans,...",https://www.pmo.gov.sg/Newsroom/National-Day-M...
3,2018-08-09,S Iswaran,National Day Message 2018,"என் சக சிங்கப்பூரர்களே, வணக்கம்.\n\nஒவ்வோர் ஆண...","My fellow Singaporeans, salute. Every year, on...",https://www.pmo.gov.sg/Newsroom/national-day-m...
4,2017-08-09,S Iswaran,National Day Message 2017,"என் சக சிங்கப்பூரர்களே,\nவணக்கம்.\n\nநான் இப்ப...","My fellow Singaporeans, salute. I am now in th...",https://www.pmo.gov.sg/Newsroom/national-day-m...


## 1.5 CHECK RESULTS

In [24]:
#uncomment to generate a separate file if you need it

#speeches_translated.to_csv('../data/tamil_to_english_speeches.csv', index=False)

In [13]:
# Check translation quality of 1 speech

speeches_translated['English_Translation'].values[3]

'My fellow Singaporeans, salute. Every year, on the ninth of August, we celebrate our progress as a nation; we pledge to be a united people. In just fifty-three years, Singapore has achieved a lot. Today, we are a vibrant and prosperous city nation. Our economy continues to see consistent growth. In recent years, that\'s 3 to 3. It has grown by 5 percent. Productivity improves; incomes rise. Singaporeans live better lives than many other nationalities. But there are signs that complications are arising. Over the past year, trade tensions between the major countries have worsened. This affects international trade, investment and industry confidence. As a result, Singapore\'s growth and progress will be affected. Trade disputes will undermine trust between major countries and impede cooperation. Therefore, regional and international security may also be at stake. For example, the situation on the Korean Peninsula. Two months ago, a high-level meeting was held between US President Donald 

# 2. TRANSLATE TAMIL STORIES TO ENGLISH

The stories on Sethi are relatively short, so we don't run into problems with the token limit. Pretty straightforward implementation 

In [14]:
stories = pd.read_csv('../data/tamil_stories.csv')

In [15]:
stories.head()

,Date,Source,Headline,Story_Text,URL
0,2020-12-23,TamilSeithi (Mediacorp),புதுவகை கொரோனா கிருமிப்பரவல் இதுவரை சிங்கப்பூர...,"சிங்கப்பூரில், புதுவகைக் கொரோனா கிருமிப்பரவல் ...",https://seithi.mediacorp.sg/mobilet/singapore/...
1,2020-12-23,TamilSeithi (Mediacorp),தாம்சன் வட்டாரத்தில் கண்டுபிடிக்கப்பட்ட போர்க்...,தாம்சன் வட்டாரத்தில் உள்ள ஜாலான் ராபு சாலையில்...,https://seithi.mediacorp.sg/mobilet/singapore/...
2,2020-12-22,TamilSeithi (Mediacorp),COVID-19: மலேசியாவில் Pfizer தடுப்பூசி போட்டுக...,மலேசியாவில் Pfizer நிறுவனத்தின் தடுப்பூசியைப் ...,https://seithi.mediacorp.sg/mobilet/asia/first...
3,2020-12-22,TamilSeithi (Mediacorp),"COVID-19: முன்னிலை ஊழியர்கள், எளிதில் பாதிக்கப...","COVID-19 தடுப்பூசிகளின் முதல் தொகுதி, சிங்கப்ப...",https://seithi.mediacorp.sg/mobilet/singapore/...
4,2020-12-22,TamilSeithi (Mediacorp),இந்தியாவில் TikTok செயலியைப் போன்ற மற்றொரு செய...,இந்தியாவில் TikTok செயலியைப் போன்ற மற்றொரு செய...,https://seithi.mediacorp.sg/mobilet/asia/india...


## 2.1 CLEAN TEXT AND TRANSLATE

In [16]:
stories["Clean_Text"] = stories['Story_Text'].map(lambda text: clean_text(text))

In [17]:
%%time
stories['Headline_Translation'] = stories["Headline"].apply(lambda text: ta2en.translate(text))

CPU times: user 30 s, sys: 76.1 ms, total: 30.1 s
Wall time: 7.64 s


In [18]:
%%time
stories['Story_Translation'] = stories["Story_Text"].apply(lambda text: ta2en.translate(text))

CPU times: user 3min 55s, sys: 1.42 s, total: 3min 56s
Wall time: 59.9 s


## 2.2 RE-ORG FOR CLARITY

In [19]:
cols2 = [
    "Date",
    "Source",
    "Headline",
    "Headline_Translation", 
    "Story_Text",
    "Story_Translation",
    "URL",
]

stories_translated = stories[cols2]


In [20]:
stories_translated.head()

,Date,Source,Headline,Headline_Translation,Story_Text,Story_Translation,URL
0,2020-12-23,TamilSeithi (Mediacorp),புதுவகை கொரோனா கிருமிப்பரவல் இதுவரை சிங்கப்பூர...,New type of corona virus hasn't been in Singap...,"சிங்கப்பூரில், புதுவகைக் கொரோனா கிருமிப்பரவல் ...","In Singapore, Education Minister Lawrence Wong...",https://seithi.mediacorp.sg/mobilet/singapore/...
1,2020-12-23,TamilSeithi (Mediacorp),தாம்சன் வட்டாரத்தில் கண்டுபிடிக்கப்பட்ட போர்க்...,A wartime bomb found in Thompson circles.,தாம்சன் வட்டாரத்தில் உள்ள ஜாலான் ராபு சாலையில்...,"On Jalan Rabu Road in Thompson locality, a war...",https://seithi.mediacorp.sg/mobilet/singapore/...
2,2020-12-22,TamilSeithi (Mediacorp),COVID-19: மலேசியாவில் Pfizer தடுப்பூசி போட்டுக...,COVID-19: Prime Minister Muhidin in the first ...,மலேசியாவில் Pfizer நிறுவனத்தின் தடுப்பூசியைப் ...,Prime Minister Muhyiddin Yassin has said he wi...,https://seithi.mediacorp.sg/mobilet/asia/first...
3,2020-12-22,TamilSeithi (Mediacorp),"COVID-19: முன்னிலை ஊழியர்கள், எளிதில் பாதிக்கப...","COVID-19: Frontline staff, susceptible people ...","COVID-19 தடுப்பூசிகளின் முதல் தொகுதி, சிங்கப்ப...","The first batch of COVID-19 vaccines, arrived ...",https://seithi.mediacorp.sg/mobilet/singapore/...
4,2020-12-22,TamilSeithi (Mediacorp),இந்தியாவில் TikTok செயலியைப் போன்ற மற்றொரு செய...,"Google, Microsoft, $100 million in funding for...",இந்தியாவில் TikTok செயலியைப் போன்ற மற்றொரு செய...,The company that developed another app similar...,https://seithi.mediacorp.sg/mobilet/asia/india...


## 2.3 CHECK RESULTS

In [21]:
#uncomment to generate a separate file if you need it

#stories_translated.to_csv('../data/tamil_to_english_stories.csv', index=False)

In [22]:
stories_translated['Story_Text'].values[1]

'தாம்சன் வட்டாரத்தில் உள்ள ஜாலான் ராபு சாலையில், போர்க்கால வெடிகுண்டு ஒன்று கண்டுபிடிக்கப்பட்டுள்ளது.\n\nதரைவீடுகள் அமைந்துள்ள அந்தப் பகுதியில் வெடிகுண்டு இருப்பதாக, நேற்று (டிசம்பர் 22) பிற்பகல் 2 மணிக்குத் தகவல் கிடைத்ததாக, சிங்கப்பூர்க் காவல்துறை தெரிவித்தது.\n\nகுடிமைத் தற்காப்புப் படையின் வெடிகுண்டு அகற்றும் பிரிவுடன் நடத்தப்பட்ட ஆலோசனையில், அதைப் பாதுகாப்பாக இடம் மாற்றி அப்புறப்படுத்த தீர்மானிக்கப்பட்டது.\n\nஇரண்டாம் உலகப் போரில் பயன்படுத்தப்பட்ட வெடிகுண்டுகள், சிங்கப்பூரில் ஆங்காங்கே அவ்வப்போது கண்டுபிடிக்கப்படுவதுண்டு.\n\nகடந்த நவம்பர் மாதம், ரிவர் வேலி பகுதியில் உள்ள ஜியாக் கிம் ரோட்டில் இரண்டாம் உலகப் போர்க்கால வெடிகுண்டு ஒன்று கட்டுப்படுத்தப்பட்ட சூழலில் வெடிக்கச் செய்யப்பட்டது.\n\nவானிலிருந்து வீசப்பட்ட 50 கிலோகிராம் வெடிகுண்டு, கட்டுமானத் தலத்தில் நிலந்தோண்டும் பணியின்போது கண்டுபிடிக்கப்பட்டது.\n\nஅதை அங்கிருந்து அப்புறப்படுத்தி எடுத்துச் செல்வது ஆபத்தானது என்பதால், கிடைத்த இடத்திலேயே அதை வெடிக்கச் செய்ய முடிவெடுக்கப்பட்டு நிறைவேற்றப்பட்டது.'

In [23]:
stories_translated['Story_Translation'].values[1]

"On Jalan Rabu Road in Thompson locality, a wartime bomb has been found.Singapore police said they received a report at 2pm yesterday (December 22) that there was a bomb in the area, where the ground houses are located.In consultation with the Civil Self-Defense Force's bomb disposal unit, it was determined to safely relocate and dispose of it.The explosives used in World War II are regularly found in Singapore.Last November, a World War II bomb was detonated in a controlled environment on Jiyak Kim Road in the River Valley area.A 50kg bomb dropped from the sky was found during landfill work at the construction site, as it was dangerous to evacuate and take it away."